# Prepatation

In [1]:
import numpy as np
import pandas as pd
import random
from sklearn import datasets

In [2]:
iris = datasets.load_iris()
species = iris.target
yesno = np.random.randint(2, size=len(species))
index = np.arange(len(species))
iris = pd.DataFrame(iris.data)
species = pd.DataFrame(species)
yesno = pd.DataFrame(yesno)
index = pd.DataFrame(index)
species = species.replace({0:'setosa', 1:'versicolor', 2:'verginica'})

# concat()
Combine multiple dataframe with the same number of rows.

In [3]:
iris = pd.concat([index.reset_index(drop=True), iris], axis=1)
iris = pd.concat([iris.reset_index(drop=True), species], axis=1)
iris = pd.concat([iris.reset_index(drop=True), yesno], axis=1)
iris.columns = ['index', 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species', 'yes_no']

In [4]:
iris.head()

,index,sepal_length,sepal_width,petal_length,petal_width,species,yes_no
0,0,5.1,3.5,1.4,0.2,setosa,0
1,1,4.9,3.0,1.4,0.2,setosa,1
2,2,4.7,3.2,1.3,0.2,setosa,0
3,3,4.6,3.1,1.5,0.2,setosa,0
4,4,5.0,3.6,1.4,0.2,setosa,0


---

# groupby()
Group the dataset by desired categorical variables and aggregate numerical variables by desired functions (e.g. mean, sd, max, min, head(1) etc...)

In [5]:
# group by categorical columns
iris_groupby = iris.groupby(['species'])['sepal_length', 'petal_length'].mean().reset_index()
iris_groupby

,species,sepal_length,petal_length
0,setosa,5.006,1.462
1,verginica,6.588,5.552
2,versicolor,5.936,4.260


In [6]:
# combine multiple columns with aggrgate fuctions (i.e. the most efficient ways to collapse multiple columns)
# even works for string (e.g. first name and last name), but probably easier just by using '+' operator.
mapping = {'sepal_length':'length',
          'sepal_width':'width',
          'petal_length':'length',
          'petal_width':'width'}
iris.set_index('index').groupby(mapping, axis=1).sum().reset_index().head()

,index,length,width
0,0,6.5,3.7
1,1,6.3,3.2
2,2,6.0,3.4
3,3,6.1,3.3
4,4,6.4,3.8


# pivot_table()
Very similar to groupby(), except we can choose multiple categorical variables. It is also used to spread a column into multiple columns.

In [7]:
iris.pivot_table(values=['sepal_length', 'petal_length'], index=['yes_no', 'species'], aggfunc=np.mean)

petal_length  sepal_length
yes_no species                               
0      setosa          1.452174      5.056522
       verginica       5.680000      6.664000
       versicolor      4.212000      5.860000
1      setosa          1.470370      4.962963
       verginica       5.424000      6.512000
       versicolor      4.308000      6.012000

In [8]:
# identical to the above one, except for the column order
iris.pivot_table(values=['sepal_length', 'petal_length'], index=['species'], aggfunc=np.mean)

,petal_length,sepal_length
species,,
setosa,1.462,5.006
verginica,5.552,6.588
versicolor,4.260,5.936


# melt()
collapse multiple columns into a new column.

In [9]:
iris_groupby

,species,sepal_length,petal_length
0,setosa,5.006,1.462
1,verginica,6.588,5.552
2,versicolor,5.936,4.260


In [10]:
# equivalent to gather() from R:tidyr
iris_melt = iris_groupby.melt(id_vars='species', var_name='sepal_petal',  value_name='length')
iris_melt

,species,sepal_petal,length
0,setosa,sepal_length,5.006
1,verginica,sepal_length,6.588
2,versicolor,sepal_length,5.936
3,setosa,petal_length,1.462
4,verginica,petal_length,5.552
5,versicolor,petal_length,4.260


To put the above table into the original form, use pivot_table() with "columns" argument. Equivalent to spread() from R:tidyr.

In [11]:
iris_melt.pivot_table(values='length', index=['species'], columns='sepal_petal')

sepal_petal,petal_length,sepal_length
species,,
setosa,1.462,5.006
verginica,5.552,6.588
versicolor,4.260,5.936


# str.split()
Split a string column into multiple columns. Useful when you have name or dates. Use groupby() to put them back.

In [15]:
iris_str = iris['species'].str.split('e', n = 1, expand = True)
iris_str = pd.concat([iris.drop('species', axis=1).reset_index(drop=True), iris_str], axis=1)
iris_str.head()

,index,sepal_length,sepal_width,petal_length,petal_width,yes_no,0,1
0,0,5.1,3.5,1.4,0.2,0,s,tosa
1,1,4.9,3.0,1.4,0.2,1,s,tosa
2,2,4.7,3.2,1.3,0.2,0,s,tosa
3,3,4.6,3.1,1.5,0.2,0,s,tosa
4,4,5.0,3.6,1.4,0.2,0,s,tosa


In [31]:
# Put the string columnns back i.e. combine
iris_str['species'] = iris_str[0] + 'e' + iris_str[1]
iris_str.drop([0,1], axis=1).head()

,index,sepal_length,sepal_width,petal_length,petal_width,yes_no,species
0,0,5.1,3.5,1.4,0.2,0,setosa
1,1,4.9,3.0,1.4,0.2,1,setosa
2,2,4.7,3.2,1.3,0.2,0,setosa
3,3,4.6,3.1,1.5,0.2,0,setosa
4,4,5.0,3.6,1.4,0.2,0,setosa


# crosstab()
Often used for a confusion matrix.

In [10]:
pd.crosstab(iris["species"],iris["yes_no"],margins=True)

yes_no,0,1,All
species,,,
setosa,26,24,50
verginica,31,19,50
versicolor,32,18,50
All,89,61,150


# One-Hot Encoding
Expand categorical variables into new variables of binaries. This is REQUIRED if one is to apply machine-learning algorithms (except for LightGBM).

In [123]:
# One way
iris_1 = pd.get_dummies(iris)
iris_1.head()

,sepal_length,sepal_width,petal_length,petal_width,species_setosa,species_verginica,species_versicolor
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


In [126]:
# Most popular way
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
iris_2 = enc.fit_transform(iris)
iris_2.toarray()

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

# merge()
Merge two distinct data files. Inner, outer, left, or right.

# apply() and vectorize()
Used when each element of columns (or a vector) is to be applied by a function.

In [94]:
iris[['sepal_length', 'petal_length']].apply(lambda l: np.square(l)).head()

,sepal_length,petal_length
0,26.01,1.96
1,24.01,1.96
2,22.09,1.69
3,21.16,2.25
4,25.00,1.96


In [100]:
sq = np.vectorize(lambda l: np.square(l))
sq(iris['sepal_length'].to_numpy())[0:5]

array([26.01, 24.01, 22.09, 21.16, 25.  ])

In [ ]:
#12/11: continue from page 82